#Sistema de Predição de Perde de Dentes


#Importando bibliotecas

In [ ]:
#Bibliotecas utilizadas
#mudar diretorio
import os
#manipulacao de dados
import pandas as pd
import numpy as np
#graficos
import matplotlib.pyplot as plt
import seaborn as sns
#metricas
from sklearn.metrics import classification_report
#classificadores selecionados
#from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#tunning e split
from sklearn.model_selection import train_test_split, GridSearchCV
#validacao cruzada
from sklearn.model_selection import cross_val_score, KFold
#padronizacao
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline
#selecao de features
from sklearn.linear_model import Lasso
#tratamento de dados faltantes
from sklearn.impute import KNNImputer

#Criação do DATAFRAME

In [ ]:
#importar datasets
#data_xpts = os.listdir('Dataset/NHANES_2013_2014/caries/') #datasets utilizados no artigo
#data_xpts.remove('OHXDEN_H.XPT')
#data_xpts.remove('OHXREF_H.XPT')
data_xpts =['https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DEMO_H.XPT',
            'https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DR1TOT_H.XPT']


In [ ]:
data_y = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/OHXDEN_H.XPT'); #y
data_y=data_y.loc[data_y.OHDEXSTS == 1]; #excluido pessoas que não fizeram o exame completo
data_y.set_index('SEQN',inplace = True);
data_y = data_y.iloc[:,35:63];#colunas de interesse 35 a 62
data_y = data_y.applymap(lambda x: x.decode('UTF-8'));
data_y.reset_index(inplace=True);
data_y.rename(columns={"index": "SEQN"},inplace=True);
data_y;

<ipython-input-94-effae71a657f>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_y = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/OHXDEN_H.XPT'); #y
<ipython-input-94-effae71a657f>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_y = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/OHXDEN_H.XPT'); #y
<ipython-input-94-effae71a657f>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all c

In [ ]:
data_y['dente_perdido'] = 0
dente_perda_check = data_y.columns[1:-1] #colunas a serem checadas
n_reg = data_y.shape[0] #numero de registros
dentes_perdidos = ["E","P","R"] #pacientes que perderam os dentes
for col in dente_perda_check:
  for i in range(n_reg):
    if data_y[col][i] in dentes_perdidos:
      data_y.dente_perdido[i] = 1;
data_y

<ipython-input-95-807ab988bf04>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_y.dente_perdido[i] = 1;


,SEQN,OHX02CTC,OHX03CTC,OHX04CTC,OHX05CTC,OHX06CTC,OHX07CTC,OHX08CTC,OHX09CTC,OHX10CTC,...,OHX23CTC,OHX24CTC,OHX25CTC,OHX26CTC,OHX27CTC,OHX28CTC,OHX29CTC,OHX30CTC,OHX31CTC,dente_perdido
0,73557.0,P,P,P,P,P,P,P,P,P,...,P,P,P,P,P,P,P,P,P,1
1,73558.0,Z,J,S,S,S,S,S,S,S,...,S,S,S,S,S,Z,S,E,E,1
2,73559.0,Z,Z,Z,S,S,S,S,Z,S,...,Z,S,S,S,S,S,Z,Z,E,1
3,73560.0,U,S,D,K,U,S,S,S,S,...,S,S,S,S,U,D,D,S,U,0
4,73561.0,E,Z,E,Z,Z,Z,R,R,Z,...,Z,R,Z,S,Z,E,E,E,E,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,83727.0,S,S,S,M,S,S,S,S,S,...,S,S,S,S,S,M,S,Z,S,0
8629,83728.0,U,U,U,D,D,D,D,D,D,...,D,D,D,D,D,D,U,U,U,0
8630,83729.0,Z,Z,E,Z,S,S,S,S,S,...,S,S,S,S,S,S,S,E,Z,1
8631,83730.0,U,S,K,K,D,D,S,S,D,...,S,S,S,D,D,K,K,S,U,0


In [ ]:
data_y.dente_perdido.value_counts()  #contagem de valores

0    5728
1    2905
Name: dente_perdido, dtype: int64

In [ ]:
data_dente_perdido = data_y.drop(columns=dente_perda_check) #excluir colunas ja transformadas
data_dente_perdido

,SEQN,dente_perdido
0,73557.0,1
1,73558.0,1
2,73559.0,1
3,73560.0,0
4,73561.0,1
...,...,...
8628,83727.0,0
8629,83728.0,0
8630,83729.0,1
8631,83730.0,0


In [ ]:
df = data_dente_perdido.copy()
for links in data_xpts: #merging
  df1 = pd.read_sas(links);
  df = pd.merge(df, df1, how = 'left', on = 'SEQN');

<ipython-input-98-80902c0ac78d>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1 = pd.read_sas(links);
<ipython-input-98-80902c0ac78d>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1 = pd.read_sas(links);
<ipython-input-98-80902c0ac78d>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1 = 

In [ ]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)
df

,SEQN,dente_perdido,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,...,DRD370QQ,DRD370R,DRD370RQ,DRD370S,DRD370SQ,DRD370T,DRD370TQ,DRD370U,DRD370UQ,DRD370V
0,73557.0,1,8.0,2.0,1.0,69.0,NaN,4.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73558.0,1,8.0,2.0,1.0,54.0,NaN,3.0,3.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
2,73559.0,1,8.0,2.0,1.0,72.0,NaN,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73560.0,0,8.0,2.0,1.0,9.0,NaN,3.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73561.0,1,8.0,2.0,2.0,73.0,NaN,3.0,3.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,83727.0,0,8.0,2.0,1.0,26.0,NaN,2.0,2.0,2.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
8629,83728.0,0,8.0,2.0,2.0,2.0,24.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8630,83729.0,1,8.0,2.0,2.0,42.0,NaN,4.0,4.0,2.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
8631,83730.0,0,8.0,2.0,1.0,7.0,NaN,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for col in df.columns:
  if ((df[col].isna().sum()) >= (int(df.shape[0]/2))):
    df.drop(columns=col,inplace=True)

df

,SEQN,dente_perdido,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,DMQMILIZ,...,DRD370M,DRD370N,DRD370O,DRD370P,DRD370Q,DRD370R,DRD370S,DRD370T,DRD370U,DRD370V
0,73557.0,1,8.0,2.0,1.0,69.0,4.0,4.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73558.0,1,8.0,2.0,1.0,54.0,3.0,3.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,73559.0,1,8.0,2.0,1.0,72.0,3.0,3.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73560.0,0,8.0,2.0,1.0,9.0,3.0,3.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73561.0,1,8.0,2.0,2.0,73.0,3.0,3.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,83727.0,0,8.0,2.0,1.0,26.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8629,83728.0,0,8.0,2.0,2.0,2.0,1.0,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8630,83729.0,1,8.0,2.0,2.0,42.0,4.0,4.0,2.0,2.0,...,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8631,83730.0,0,8.0,2.0,1.0,7.0,2.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Pré-processamento


##Remoção de Outliers e OneHotEncoder

In [ ]:
#Removendo outliers
#DBD100 - 7,9
#DRQSPREP - 9
#DR1STY - 9
#DRQSDIET - 9
#DR1_300 - 7,9
#DRD340 - 7,9
#DRD360 - 7,9
#DMQMILIZ  - 7,9
#DMDBORN4 - 77,99
#DMDCITZN -7,9
outliers_7_9 = ['DBD100','DR1_300','DRD340','DRD360','DMQMILIZ','DMDCITZN',
                'DMDEDUC2','DMDHREDU','DMDHSEDU','DBD100','DRQSPREP',]
outliers_9 = ['DRQSPREP','DR1STY','DRQSDIET']
outliers_77_99 =['DMDBORN4','DMDMARTL','DMDHRBR4','DMDHRMAR','INDHHIN2',
                 'INDFMIN2','DR1MNRSP','DR1HELPD']
outliers_99 = ['DBQ095Z','DR1TWS']
for i in range(df.shape[0]):
  for col in outliers_7_9:
    if ((df[col][i] >= 7)):
      df[col][i] = np.nan;
  for col in outliers_9:
    if ((df[col][i] == 9)):
      df[col][i] = np.nan;
  for col in outliers_77_99:
    if ((df[col][i] >= 77)):
      df[col][i] = np.nan;
  for col in outliers_99:
    if ((df[col][i] == 99)):
      df[col][i] = np.nan;
df;

<ipython-input-101-dbcc0d1232cc>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][i] = np.nan;
<ipython-input-101-dbcc0d1232cc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][i] = np.nan;
<ipython-input-101-dbcc0d1232cc>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][i] = np.nan;
<ipython-input-101-dbcc0d1232cc>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [ ]:
#ONE HOT ENCODER
dummies=['DR1DRSTZ','DR1DAY','DR1LANG','DR1MNRSP','DR1HELPD','DBQ095Z','DR1TWS',
         'RIDRETH1','RIDRETH3','DMDEDUC2','DMDMARTL','AIALANGA','DMDHRBR4',
         'DMDHREDU','DMDHRMAR','DMDHSEDU','INDHHIN2','INDFMIN2']
df_dummy = df[dummies].astype(str);
df_dummy2 = pd.get_dummies(df_dummy, dtype=float,dummy_na=False);


In [ ]:
nan_columns = []
for col in df_dummy2.columns:
  if col.endswith('_nan'):
    nan_columns.append(col)
nan_columns

['DR1DAY_nan',
 'DR1LANG_nan',
 'DR1MNRSP_nan',
 'DR1HELPD_nan',
 'DBQ095Z_nan',
 'DR1TWS_nan',
 'DMDEDUC2_nan',
 'DMDMARTL_nan',
 'AIALANGA_nan',
 'DMDHRBR4_nan',
 'DMDHREDU_nan',
 'DMDHRMAR_nan',
 'DMDHSEDU_nan',
 'INDHHIN2_nan',
 'INDFMIN2_nan']

In [ ]:
df=pd.concat([df,df_dummy2],axis=1)
df.drop(columns=dummies,inplace=True)
df.drop(columns=nan_columns,inplace=True)

##Normalização

In [ ]:
df_c = df.copy()
colunas_df = df.columns
df.drop(columns=['SEQN'], inplace=True)

y = df[['dente_perdido']]
X = df.drop(columns=['dente_perdido'])
X_c = X.copy()
colunas_X = X.columns

# define standard scaler
scaler = StandardScaler()

# transform data
X = scaler.fit_transform(X)

X = pd.DataFrame(X,columns = colunas_X )

In [ ]:
#divisão da base
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.3, random_state=0)
X_train

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDEXMON,DMQMILIZ,DMDBORN4,DMDCITZN,SIALANG,SIAPROXY,...,INDFMIN2_14.0,INDFMIN2_15.0,INDFMIN2_2.0,INDFMIN2_3.0,INDFMIN2_4.0,INDFMIN2_5.0,INDFMIN2_6.0,INDFMIN2_7.0,INDFMIN2_8.0,INDFMIN2_9.0
3780,0.0,0.0,-1.001507,0.078503,0.981066,0.301172,-0.480544,-0.32286,-0.34094,0.753231,...,-0.30073,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908
3887,0.0,0.0,0.998495,1.099997,-1.019299,0.301172,-0.480544,-0.32286,-0.34094,0.753231,...,-0.30073,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908
2015,0.0,0.0,-1.001507,-0.985553,0.981066,NaN,-0.480544,-0.32286,-0.34094,-1.327614,...,-0.30073,-0.452085,4.560757,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908
1742,0.0,0.0,0.998495,0.887186,-1.019299,0.301172,-0.480544,-0.32286,-0.34094,0.753231,...,-0.30073,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,2.733502,-0.31204,-0.282967,-0.230908
7196,0.0,0.0,-1.001507,-0.985553,0.981066,NaN,-0.480544,-0.32286,-0.34094,-1.327614,...,-0.30073,-0.452085,-0.219262,-0.266132,-0.271581,3.239651,-0.365831,-0.31204,-0.282967,-0.230908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6446,0.0,0.0,0.998495,-1.198364,0.981066,NaN,-0.480544,-0.32286,-0.34094,-1.327614,...,-0.30073,-0.452085,-0.219262,-0.266132,-0.271581,3.239651,-0.365831,-0.31204,-0.282967,-0.230908
2333,0.0,0.0,0.998495,-0.942990,-1.019299,NaN,-0.480544,-0.32286,-0.34094,-1.327614,...,-0.30073,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,3.533987,-0.230908
8369,0.0,0.0,0.998495,1.099997,-1.019299,0.301172,-0.480544,-0.32286,-0.34094,0.753231,...,-0.30073,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,2.733502,-0.31204,-0.282967,-0.230908
3593,0.0,0.0,0.998495,0.121065,-1.019299,0.301172,-0.480544,-0.32286,-0.34094,0.753231,...,-0.30073,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,3.533987,-0.230908


##Tratamento de Dados Faltantes

In [ ]:
df_train =  pd.merge(X_train, y_train, left_index=True, right_index=True);
df_train.head()
df_train_c = df_train.copy()

In [ ]:
imputer = KNNImputer(n_neighbors=3)
df_train_knn = imputer.fit_transform(df_train)
df_train_knn


array([[ 0.        ,  0.        , -1.00150699, ..., -0.28296654,
        -0.23090841,  0.        ],
       [ 0.        ,  0.        ,  0.99849528, ..., -0.28296654,
        -0.23090841,  0.        ],
       [ 0.        ,  0.        , -1.00150699, ..., -0.28296654,
        -0.23090841,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.99849528, ..., -0.28296654,
        -0.23090841,  1.        ],
       [ 0.        ,  0.        ,  0.99849528, ...,  3.53398677,
        -0.23090841,  0.        ],
       [ 0.        ,  0.        ,  0.99849528, ..., -0.28296654,
        -0.23090841,  0.        ]])

In [ ]:
#substituindo valores tratados
for i in range(df_train.shape[0]):
  df_train.iloc[i,:] = df_train_knn[i]

In [ ]:
df_train.head()

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDEXMON,DMQMILIZ,DMDBORN4,DMDCITZN,SIALANG,SIAPROXY,...,INDFMIN2_15.0,INDFMIN2_2.0,INDFMIN2_3.0,INDFMIN2_4.0,INDFMIN2_5.0,INDFMIN2_6.0,INDFMIN2_7.0,INDFMIN2_8.0,INDFMIN2_9.0,dente_perdido
3780,0.0,0.0,-1.001507,0.078503,0.981066,0.301172,-0.480544,-0.32286,-0.34094,0.753231,...,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908,0
3887,0.0,0.0,0.998495,1.099997,-1.019299,0.301172,-0.480544,-0.32286,-0.34094,0.753231,...,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908,0
2015,0.0,0.0,-1.001507,-0.985553,0.981066,0.301172,-0.480544,-0.32286,-0.34094,-1.327614,...,-0.452085,4.560757,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908,0
1742,0.0,0.0,0.998495,0.887186,-1.019299,0.301172,-0.480544,-0.32286,-0.34094,0.753231,...,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,2.733502,-0.31204,-0.282967,-0.230908,1
7196,0.0,0.0,-1.001507,-0.985553,0.981066,0.301172,-0.480544,-0.32286,-0.34094,-1.327614,...,-0.452085,-0.219262,-0.266132,-0.271581,3.239651,-0.365831,-0.31204,-0.282967,-0.230908,0


In [ ]:
y_train = df_train[['dente_perdido']]
X_train =  df_train.drop(columns=['dente_perdido'])
y_train.value_counts()


dente_perdido
0                4010
1                2033
dtype: int64

In [ ]:
df_test = pd.merge(X_test, y_test, left_index=True, right_index=True)
df_test

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDEXMON,DMQMILIZ,DMDBORN4,DMDCITZN,SIALANG,SIAPROXY,...,INDFMIN2_15.0,INDFMIN2_2.0,INDFMIN2_3.0,INDFMIN2_4.0,INDFMIN2_5.0,INDFMIN2_6.0,INDFMIN2_7.0,INDFMIN2_8.0,INDFMIN2_9.0,dente_perdido
8346,0.0,0.0,0.998495,-0.645055,-1.019299,NaN,-0.480544,-0.32286,2.933069,0.753231,...,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908,0
4716,0.0,0.0,0.998495,1.312808,0.981066,0.301172,2.080974,-0.32286,-0.340940,0.753231,...,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,2.733502,-0.31204,-0.282967,-0.230908,1
8553,0.0,0.0,0.998495,-0.687617,-1.019299,NaN,-0.480544,-0.32286,-0.340940,-1.327614,...,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,3.533987,-0.230908,0
6864,0.0,0.0,-1.001507,0.333877,-1.019299,0.301172,-0.480544,-0.32286,-0.340940,0.753231,...,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,2.733502,-0.31204,-0.282967,-0.230908,1
980,0.0,0.0,0.998495,-1.283488,0.981066,NaN,-0.480544,-0.32286,-0.340940,-1.327614,...,-0.452085,-0.219262,3.757538,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,0.0,0.0,0.998495,-0.857866,-1.019299,NaN,-0.480544,-0.32286,-0.340940,-1.327614,...,2.211975,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908,0
4336,0.0,0.0,0.998495,1.270246,0.981066,0.301172,-0.480544,-0.32286,-0.340940,0.753231,...,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908,0
3640,0.0,0.0,0.998495,0.121065,-1.019299,0.301172,-0.480544,-0.32286,-0.340940,0.753231,...,-0.452085,-0.219262,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.282967,-0.230908,1
2835,0.0,0.0,-1.001507,0.121065,0.981066,0.301172,-0.480544,-0.32286,-0.340940,0.753231,...,-0.452085,-0.219262,-0.266132,-0.271581,3.239651,-0.365831,-0.31204,-0.282967,-0.230908,1


##LASSO - feature selection

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV(cv=5, random_state=0).fit(X_train, y_train);
reg.score(X_train, y_train);

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
coefs_lasso = reg.coef_

In [ ]:
colunas = df.columns
# colunas_X
i = 0;
features_names = []
for coef in coefs_lasso:
  if coef != 0:
    features_names.append(colunas[i])
  i += 1
print(len(features_names))
features_names


98


['RIAGENDR',
 'RIDEXMON',
 'DMQMILIZ',
 'DMDBORN4',
 'DMDFMSIZ',
 'DMDHHSZA',
 'WTINT2YR',
 'SDMVSTRA',
 'WTDRD1',
 'DRABF',
 'DR1DBIH',
 'DRQSPREP',
 'DR1STY',
 'DR1TPROT',
 'DR1TCARB',
 'DR1TSUGR',
 'DR1TCHOL',
 'DR1TATOA',
 'DR1TCRYP',
 'DR1TLYCO',
 'DR1TFA',
 'DR1TB12A',
 'DR1TVC',
 'DR1TZINC',
 'DR1TSELE',
 'DR1TCAFF',
 'DR1TTHEO',
 'DR1TM181',
 'DR1TM201',
 'DR1TP183',
 'DR1_320Z',
 'DR1BWATZ',
 'DRD340',
 'DRD360',
 'DRD370E',
 'DRD370F',
 'DRD370G',
 'DRD370H',
 'DRD370I',
 'DRD370L',
 'DRD370M',
 'DRD370N',
 'DRD370Q',
 'DRD370S',
 'DRD370T',
 'DR1DAY_2.0',
 'DR1DAY_3.0',
 'DR1DAY_4.0',
 'DR1DAY_6.0',
 'DR1LANG_1.0',
 'DR1LANG_2.0',
 'DR1LANG_4.0',
 'DR1LANG_5.0',
 'DR1LANG_6.0',
 'DR1MNRSP_1.0',
 'DR1MNRSP_12.0',
 'DR1MNRSP_2.0',
 'DR1MNRSP_3.0',
 'DR1HELPD_10.0',
 'DR1HELPD_3.0',
 'DR1HELPD_4.0',
 'DR1HELPD_6.0',
 'DR1HELPD_8.0',
 'DR1TWS_4.0',
 'RIDRETH1_3.0',
 'RIDRETH1_5.0',
 'RIDRETH3_3.0',
 'RIDRETH3_6.0',
 'RIDRETH3_7.0',
 'DMDEDUC2_1.0',
 'DMDEDUC2_2.0',
 'DMDEDUC2_3.

In [ ]:
X_train_lasso = X_train[features_names]
X_test_lasso = X_test[features_names]

In [ ]:
X_train_lasso

,RIAGENDR,RIDEXMON,DMQMILIZ,DMDBORN4,DMDFMSIZ,DMDHHSZA,WTINT2YR,SDMVSTRA,WTDRD1,DRABF,...,INDHHIN2_10.0,INDHHIN2_2.0,INDHHIN2_7.0,INDFMIN2_1.0,INDFMIN2_12.0,INDFMIN2_13.0,INDFMIN2_15.0,INDFMIN2_3.0,INDFMIN2_7.0,INDFMIN2_8.0
3780,-1.001507,0.981066,0.301172,-0.480544,0.699975,0.617551,0.822005,-0.218428,0.034096,0.051028,...,5.047930,-0.204641,-0.311369,-0.198101,-0.147982,-0.107706,-0.452085,-0.266132,-0.31204,-0.282967
3887,0.998495,-1.019299,0.301172,-0.480544,-0.992236,-0.644598,-0.592256,1.669388,-0.518609,0.051028,...,-0.198101,-0.204641,-0.311369,-0.198101,-0.147982,9.284504,-0.452085,-0.266132,-0.31204,-0.282967
2015,-1.001507,0.981066,0.301172,-0.480544,0.699975,0.617551,-0.498259,0.489503,-0.621284,0.051028,...,-0.198101,4.886611,-0.311369,-0.198101,-0.147982,-0.107706,-0.452085,-0.266132,-0.31204,-0.282967
1742,0.998495,-1.019299,0.301172,-0.480544,1.828115,-0.644598,-0.277790,1.197434,-0.281715,0.051028,...,-0.198101,-0.204641,-0.311369,-0.198101,-0.147982,-0.107706,-0.452085,-0.266132,-0.31204,-0.282967
7196,-1.001507,0.981066,0.301172,-0.480544,1.828115,0.617551,-0.859723,-0.690383,-0.346035,0.051028,...,-0.198101,-0.204641,-0.311369,-0.198101,-0.147982,-0.107706,-0.452085,-0.266132,-0.31204,-0.282967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6446,0.998495,0.981066,0.301172,-0.480544,1.264045,1.879701,-0.561546,-0.454406,-0.602258,0.051028,...,-0.198101,-0.204641,-0.311369,-0.198101,-0.147982,-0.107706,-0.452085,-0.266132,-0.31204,-0.282967
2333,0.998495,-1.019299,0.301172,-0.480544,1.264045,-0.644598,-0.683713,1.197434,0.041909,0.051028,...,-0.198101,-0.204641,-0.311369,-0.198101,-0.147982,-0.107706,-0.452085,-0.266132,-0.31204,3.533987
8369,0.998495,-1.019299,0.301172,-0.480544,-0.992236,-0.644598,-0.230479,0.961457,0.231199,0.051028,...,-0.198101,-0.204641,-0.311369,-0.198101,-0.147982,-0.107706,-0.452085,-0.266132,-0.31204,-0.282967
3593,0.998495,-1.019299,0.301172,-0.480544,0.135905,0.617551,-0.222438,1.433411,-0.409355,0.051028,...,-0.198101,-0.204641,-0.311369,-0.198101,-0.147982,-0.107706,-0.452085,-0.266132,-0.31204,3.533987


##SMOTE

In [ ]:
# Oversample and plot imbalanced dataset with SMOTE
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from matplotlib import pyplot
from numpy import where
# transform the dataset
oversample = SMOTE()
undersample = RandomUnderSampler(random_state=0)
X_train_O, y_train_O = oversample.fit_resample(X_train_lasso, y_train)
X_train_U, y_train_U = undersample.fit_resample(X_train_lasso, y_train)

#Modelos

In [ ]:
#classificadores selecionados
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

In [ ]:
#OVERSAMPLE
logreg = LogisticRegression(random_state=0,max_iter=1000,verbose = False).fit(X_train_O, y_train_O);
svc = SVC(gamma='auto',verbose = False).fit(X_train_O, y_train_O);
arv = DecisionTreeClassifier(random_state=0).fit(X_train_O, y_train_O);
rfor = RandomForestClassifier(random_state=0).fit(X_train_O, y_train_O);
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train_O, y_train_O);
naive = BernoulliNB(force_alpha=True).fit(X_train_O, y_train_O);
#Predições
df_test =  pd.merge(X_test_lasso,y_test, left_index = True, right_index = True);
df_test = df_test.dropna(how='any');
X_test2 = df_test.drop(columns = ['dente_perdido']);
y_test = df_test[['dente_perdido']];
y_pred_logreg = logreg.predict(X_test2);
y_pred_svc = svc.predict(X_test2);
y_pred_arv = arv.predict(X_test2);
y_pred_rfor = rfor.predict(X_test2);
y_pred_knn = knn.predict(X_test2);
y_pred_naive = naive.predict(X_test2);

print("\nREGRESSÃO LOGÍSITCA\n")
print(classification_report(y_test, y_pred_logreg))
print("\nSVM\n")
print(classification_report(y_test, y_pred_svc))
print("\nÁRVORE DE DECISÃO\n")
print(classification_report(y_test, y_pred_arv))
print("\nRANDOM FOREST\n")
print(classification_report(y_test, y_pred_rfor))
print("\nKNN\n")
print(classification_report(y_test, y_pred_knn))
print("\nNAIVE-BAYES\n")
print(classification_report(y_test, y_pred_naive))



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-120-0f2c0f3bc191>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfor = RandomForestClassifier(random_state=0).fit(X_train_O, y_train_O);
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,)


REGRESSÃO LOGÍSITCA

              precision    recall  f1-score   support

           0       0.78      0.54      0.64       430
           1       0.71      0.88      0.78       541

    accuracy                           0.73       971
   macro avg       0.74      0.71      0.71       971
weighted avg       0.74      0.73      0.72       971


SVM

              precision    recall  f1-score   support

           0       0.80      0.55      0.65       430
           1       0.71      0.89      0.79       541

    accuracy                           0.74       971
   macro avg       0.76      0.72      0.72       971
weighted avg       0.75      0.74      0.73       971


ÁRVORE DE DECISÃO

              precision    recall  f1-score   support

           0       0.60      0.66      0.63       430
           1       0.71      0.66      0.68       541

    accuracy                           0.66       971
   macro avg       0.66      0.66      0.66       971
weighted avg       0.66   

In [ ]:
#UNDERSAMPLE
logreg = LogisticRegression(random_state=0,max_iter=1000,verbose = False).fit(X_train_U, y_train_U);
svc = SVC(gamma='auto',verbose = False).fit(X_train_U, y_train_U);
arv = DecisionTreeClassifier(random_state=0).fit(X_train_U, y_train_U);
rfor = RandomForestClassifier(random_state=0).fit(X_train_U, y_train_U);
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train_U, y_train_U);
naive = BernoulliNB(force_alpha=True).fit(X_train_U, y_train_U);
#Predições
df_test =  pd.merge(X_test_lasso,y_test, left_index = True, right_index = True);
df_test = df_test.dropna(how='any');
X_test2 = df_test.drop(columns = ['dente_perdido']);
y_test = df_test[['dente_perdido']];
y_pred_logreg = logreg.predict(X_test2);
y_pred_svc = svc.predict(X_test2);
y_pred_arv = arv.predict(X_test2);
y_pred_rfor = rfor.predict(X_test2);
y_pred_knn = knn.predict(X_test2);
y_pred_naive = naive.predict(X_test2);


print("\nREGRESSÃO LOGÍSITCA\n")
print(classification_report(y_test, y_pred_logreg))
print("\nSVM\n")
print(classification_report(y_test, y_pred_svc))
print("\nÁRVORE DE DECISÃO\n")
print(classification_report(y_test, y_pred_arv))
print("\nRANDOM FOREST\n")
print(classification_report(y_test, y_pred_rfor))
print("\nKNN\n")
print(classification_report(y_test, y_pred_knn))
print("\nNAIVE-BAYES\n")
print(classification_report(y_test, y_pred_naive))



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-121-6d92fec6b58c>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfor = RandomForestClassifier(random_state=0).fit(X_train_U, y_train_U);
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,)


REGRESSÃO LOGÍSITCA

              precision    recall  f1-score   support

           0       0.78      0.51      0.62       430
           1       0.70      0.89      0.78       541

    accuracy                           0.72       971
   macro avg       0.74      0.70      0.70       971
weighted avg       0.73      0.72      0.71       971


SVM

              precision    recall  f1-score   support

           0       0.84      0.49      0.61       430
           1       0.69      0.92      0.79       541

    accuracy                           0.73       971
   macro avg       0.76      0.71      0.70       971
weighted avg       0.76      0.73      0.71       971


ÁRVORE DE DECISÃO

              precision    recall  f1-score   support

           0       0.60      0.51      0.55       430
           1       0.65      0.73      0.69       541

    accuracy                           0.63       971
   macro avg       0.63      0.62      0.62       971
weighted avg       0.63   

##Modelos sem SMOTE

In [ ]:
logreg = LogisticRegression(random_state=0,max_iter=1000,verbose = False).fit(X_train_lasso, y_train);
svc = SVC(gamma='auto',verbose = False).fit(X_train_lasso, y_train);
arv = DecisionTreeClassifier(random_state=0).fit(X_train_lasso, y_train);
rfor = RandomForestClassifier(random_state=0).fit(X_train_lasso, y_train);
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train_lasso, y_train);
naive = BernoulliNB(force_alpha=True).fit(X_train_lasso, y_train);
#Predições
df_test =  pd.merge(X_test_lasso,y_test, left_index = True, right_index = True);
df_test = df_test.dropna(how='any');
X_test2 = df_test.drop(columns = ['dente_perdido']);
y_test = df_test[['dente_perdido']];
y_pred_logreg = logreg.predict(X_test2);
y_pred_svc = svc.predict(X_test2);
y_pred_arv = arv.predict(X_test2);
y_pred_rfor = rfor.predict(X_test2);
y_pred_knn = knn.predict(X_test2);
y_pred_naive = naive.predict(X_test2);


print("\nREGRESSÃO LOGÍSITCA\n")
print(classification_report(y_test, y_pred_logreg))
print("\nSVM\n")
print(classification_report(y_test, y_pred_svc))
print("\nÁRVORE DE DECISÃO\n")
print(classification_report(y_test, y_pred_arv))
print("\nRANDOM FOREST\n")
print(classification_report(y_test, y_pred_rfor))
print("\nKNN\n")
print(classification_report(y_test, y_pred_knn))
print("\nNAIVE-BAYES\n")
print(classification_report(y_test, y_pred_naive))



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-122-35351ef5ca7e>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfor = RandomForestClassifier(random_state=0).fit(X_train_lasso, y_train);
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples


REGRESSÃO LOGÍSITCA

              precision    recall  f1-score   support

           0       0.72      0.73      0.73       430
           1       0.78      0.78      0.78       541

    accuracy                           0.76       971
   macro avg       0.75      0.75      0.75       971
weighted avg       0.76      0.76      0.76       971


SVM

              precision    recall  f1-score   support

           0       0.69      0.71      0.70       430
           1       0.76      0.75      0.76       541

    accuracy                           0.73       971
   macro avg       0.73      0.73      0.73       971
weighted avg       0.73      0.73      0.73       971


ÁRVORE DE DECISÃO

              precision    recall  f1-score   support

           0       0.59      0.63      0.61       430
           1       0.69      0.65      0.67       541

    accuracy                           0.64       971
   macro avg       0.64      0.64      0.64       971
weighted avg       0.64   

##SEM SMOTE E LASSO

In [ ]:
logreg = LogisticRegression(random_state=0,max_iter=1000,verbose = False).fit(X_train, y_train);
svc = SVC(gamma='auto',verbose = False).fit(X_train, y_train);
arv = DecisionTreeClassifier(random_state=0).fit(X_train, y_train);
rfor = RandomForestClassifier(random_state=0).fit(X_train, y_train);
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train);
naive = BernoulliNB(force_alpha=True).fit(X_train, y_train);
#Predições
df_test =  pd.merge(X_test,y_test, left_index = True, right_index = True);
df_test = df_test.dropna(how='any');
X_test2 = df_test.drop(columns = ['dente_perdido']);
y_test = df_test[['dente_perdido']];
y_pred_logreg = logreg.predict(X_test2);
y_pred_svc = svc.predict(X_test2);
y_pred_arv = arv.predict(X_test2);
y_pred_rfor = rfor.predict(X_test2);
y_pred_knn = knn.predict(X_test2);
y_pred_naive = naive.predict(X_test2);


print("\nREGRESSÃO LOGÍSITCA\n")
print(classification_report(y_test, y_pred_logreg))
print("\nSVM\n")
print(classification_report(y_test, y_pred_svc))
print("\nÁRVORE DE DECISÃO\n")
print(classification_report(y_test, y_pred_arv))
print("\nRANDOM FOREST\n")
print(classification_report(y_test, y_pred_rfor))
print("\nKNN\n")
print(classification_report(y_test, y_pred_knn))
print("\nNAIVE-BAYES\n")
print(classification_report(y_test, y_pred_naive))


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-123-d6075d3b4736>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfor = RandomForestClassifier(random_state=0).fit(X_train, y_train);
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), fo


REGRESSÃO LOGÍSITCA

              precision    recall  f1-score   support

           0       0.79      0.81      0.80       288
           1       0.84      0.82      0.83       340

    accuracy                           0.82       628
   macro avg       0.81      0.81      0.81       628
weighted avg       0.82      0.82      0.82       628


SVM

              precision    recall  f1-score   support

           0       0.77      0.80      0.78       288
           1       0.82      0.80      0.81       340

    accuracy                           0.80       628
   macro avg       0.80      0.80      0.80       628
weighted avg       0.80      0.80      0.80       628


ÁRVORE DE DECISÃO

              precision    recall  f1-score   support

           0       0.67      0.70      0.68       288
           1       0.73      0.71      0.72       340

    accuracy                           0.70       628
   macro avg       0.70      0.70      0.70       628
weighted avg       0.71   

**MODELO ESCOLHIDO:** Devido ao melhor desempenho, o modelo escolhido foi o da regressão logística. Além de ter performado melhor, este é um dos mais simples e possui uma boa explicabilidade. A aplicação do SMOTE melhorou o desempenho do modelo, porém não de maneira significativa, diferente da regressão LASSO.

Tuning do Modelo Escolhido

In [ ]:
#identificar melhor função de ativação e melhor solver
parameters = {'C':(1,2.5,5,7.5,10),
              'solver':('lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga')}
logreg = LogisticRegression(random_state=0,max_iter=3000,verbose = False);
modelo = GridSearchCV(logreg, parameters, cv = 5);
modelo.fit(X_train_O, y_train_O);


In [ ]:
print(modelo.best_params_)

{'C': 1, 'solver': 'lbfgs'}


**MELHORES FEATURES - RANDOM FOREST**

In [ ]:
rfor = RandomForestClassifier(random_state=0).fit(X_train, y_train);
#Predições
df_test =  pd.merge(X_test,y_test, left_index = True, right_index = True);
df_test = df_test.dropna(how='any');
X_test2 = df_test.drop(columns = ['dente_perdido']);
y_test = df_test[['dente_perdido']];

y_pred_rfor = rfor.predict(X_test2);
melhores_previsores = rfor.feature_importances_



In [ ]:
colunas = X_train.columns

previrores_l = []
i = 0;
for previsor in melhores_previsores:
  previrores_l.append([colunas[i],previsor])
  i+=1;
df_previsores = pd.DataFrame(previrores_l,columns=['Previsor','Importância'])
df_previsores.sort_values(by='Importância',inplace=True,ascending=False)
df_previsores.reset_index(drop=True,inplace=True)
df_previsores.head(10)

,Previsor,Importância
0,RIDAGEYR,0.148427
1,DMDHRAGE,0.044580
2,SIAPROXY,0.039029
3,DMDHHSZE,0.033267
4,DR1TCAFF,0.024699
5,DMDMARTL_1.0,0.022366
6,DMDHHSIZ,0.018168
7,DMDFMSIZ,0.016579
8,DMDHHSZB,0.014449
9,DMDEDUC2_3.0,0.012918


**RIDAGEYR** – Idade em anos,
DMDHRAGE – Idade do responsável,
**SIAPROXY** – Usou ajuda de terceiros na entrevista sobre si mesmo,
DMDHHSZE - Número de idosos com mais de 60 anos na casa,
**DR1TCAFF** – Cafeína (microgramas)
**DMDMARTL_1.0** - estado civil de pessoas casadas
**DMDHHSIZ** - Número total de pessoas na casa
DMDFMSIZ - Número total de pessoas na família
DMDHHSZB - Número total de crianças de 6 a 17 anos em casa
**DMDEDUC2_3.0** - Alunos da terceira série